<a href="https://colab.research.google.com/github/mostafa-ja/Data-Mining-fall-2023/blob/main/accepted_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install xgboost
!pip install seaborn

# Download data

In [2]:
import gdown

file_id = '1quMzXrCUCVTHaVTAT26jO-T_HnhSBxtp'

download_link = f'https://drive.google.com/uc?id={file_id}'
output = 'EX_2.zip'
gdown.download(download_link, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1quMzXrCUCVTHaVTAT26jO-T_HnhSBxtp
To: /content/EX_2.zip
100%|██████████| 125M/125M [00:02<00:00, 51.3MB/s]


'EX_2.zip'

In [3]:
!unzip '/content/EX_2.zip' -d "/content/data/"

Archive:  /content/EX_2.zip
  inflating: /content/data/Answer.rar  
  inflating: /content/data/Q_A.rar   
  inflating: /content/data/Questions.rar  
  inflating: /content/data/U.rar     
  inflating: /content/data/tags.rar  
  inflating: /content/data/user_badge.rar  


In [4]:
!unrar x "/content/data/*.rar" "/content/data"


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from /content/data/Q_A.rar

Extracting  /content/data/Q_A.csv                                          3%  6% 10% 11%  OK 

Extracting from /content/data/Answer.rar

Extracting  /content/data/Answer.csv                                      14% 18% 21% 24% 27%  OK 

Extracting from /content/data/Questions.rar

Extracting  /content/data/Questions.txt                                   31% 34% 37%  OK 

Extracting from /content/data/U.rar

Extracting  /content/data/U.csv                                           40%  OK 

Extracting from /content/data/tags.rar

Extracting  /content/data/tags.txt                                        40% 41% 42% 43% 44% 45% 46% 47% 48% 49%  OK 

Extracting from /content/data/user_badge.rar

Extracting  /content/data/user_badge.txt                          

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Load datasets

In [6]:
answers = pd.read_csv('data/Answer.csv', encoding="utf-16")
answers = answers.drop(0) # remove the meaningless row(the second line)
answers.head(10)

,Id,CreationDate,Score,OwnerUserId,CommentCount,ClosedDate
1,127,2008-08-01 16:13:47.600000000,37.0,35.0,2.0,NaN
2,128,2008-08-01 16:19:04.283000000,7.0,26.0,2.0,NaN
3,133,2008-08-01 16:31:43.430000000,0.0,59.0,1.0,NaN
4,139,2008-08-01 16:43:30.880000000,5.0,72.0,0.0,NaN
5,142,2008-08-01 16:56:55.830000000,2.0,42.0,0.0,NaN
6,143,2008-08-01 17:01:27.240000000,3.0,86.0,0.0,NaN
7,169,2008-08-01 18:23:59.040000000,30.0,93.0,0.0,NaN
8,183,2008-08-01 18:51:12.090000000,35.0,86.0,0.0,NaN
9,186,2008-08-01 19:07:59.480000000,2.0,40.0,0.0,NaN
10,190,2008-08-01 19:21:57.517000000,4.0,55.0,0.0,NaN


In [7]:
len(answers['ClosedDate'].dropna())

0

In [8]:
questions = pd.read_csv('data/Questions.txt', encoding="utf-16")
questions.head(3)

,Id,CreationDate,Score,ViewCount,OwnerUserId,CommentCount,ClosedDate,FavoriteCount
0,123,2008-08-01 16:08:52,48,44161,78.0,3,NaN,13
1,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2
2,382,2008-08-02 08:58:28,28,6827,122.0,1,NaN,4


In [9]:
Q_A = pd.read_csv('data/Q_A.csv', encoding="utf-16")
Q_A.head(3)

,QId,AId,accepted
0,898405,1153025,0
1,12679343,12679433,0
2,19108614,19786664,0


In [10]:
Q_A[Q_A['accepted']==1]

,QId,AId,accepted
7,9371288,9371326,1
8,23613344,23613857,1
9,24660822,24702541,1
10,6859287,6860105,1
18,13988691,13990370,1
...,...,...,...
1597492,14586647,14612066,1
1597498,16558302,16558462,1
1597504,14422160,14422297,1
1597506,25098589,25098679,1


In [11]:
users = pd.read_csv('data/U.csv', encoding="utf-16")
users.head(3)

,Id,Reputation,Views,UpVotes,DownVotes
0,1,33107,173080,3225,1299
1,3,10145,18988,5654,100
2,9,8716,3152,42,4


In [12]:
user_badge = pd.read_csv('data/user_badge.txt', encoding="utf-16")
user_badge.head(10)

,UserId,Name
0,3718,Teacher
1,994,Teacher
2,3893,Teacher
3,4591,Teacher
4,5196,Teacher
5,2635,Teacher
6,1113,Teacher
7,4182,Teacher
8,164,Teacher
9,652,Teacher


In [13]:
tags = pd.read_csv('data/tags.txt', sep='\t')
tags.head(10)

,Id,Tags
0,2288942,user-interface
1,4433442,canvas
2,6381190,shortcut
3,27466868,jboss7.x
4,8375328,math
5,9530883,cdi
6,14851537,jdbc
7,16153458,eclipse
8,4400311,go
9,7001897,viewflipper


In [14]:
len(questions)

810071

In [15]:
len(answers)

1510812

In [16]:
merged_A_Q = pd.merge(answers, Q_A, left_on='Id', right_on='AId', how='inner')
merged_A_Q = pd.merge(merged_A_Q, questions, left_on='QId', right_on='Id', how='inner',suffixes=('_A', '_Q'))
merged_A_Q.head()

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,CreationDate_Q,Score_Q,ViewCount,OwnerUserId_Q,CommentCount_Q,ClosedDate_Q,FavoriteCount
0,127,2008-08-01 16:13:47.600000000,37.0,35.0,2.0,NaN,126,127,1,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2
1,128,2008-08-01 16:19:04.283000000,7.0,26.0,2.0,NaN,126,128,0,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2
2,139,2008-08-01 16:43:30.880000000,5.0,72.0,0.0,NaN,126,139,0,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2
3,142,2008-08-01 16:56:55.830000000,2.0,42.0,0.0,NaN,126,142,0,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2
4,143,2008-08-01 17:01:27.240000000,3.0,86.0,0.0,NaN,126,143,0,126,2008-08-01 16:10:30,33,6357,58.0,0,2012-05-08 18:11:27,2


In [17]:
len(merged_A_Q)

1510812

In [18]:
merged_A_Q_U = pd.merge(merged_A_Q, users,  left_on='OwnerUserId_A', right_on='Id', how='inner')
merged_A_Q_U.head()

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,ViewCount,OwnerUserId_Q,CommentCount_Q,ClosedDate_Q,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes
0,127,2008-08-01 16:13:47.600000000,37.0,35.0,2.0,NaN,126,127,1,126,...,6357,58.0,0,2012-05-08 18:11:27,2,35,11543,1060,436,208
1,3886,2008-08-06 19:29:24.427000000,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,61344,122.0,3,NaN,72,35,11543,1060,436,208
2,11567,2008-08-14 19:26:35.570000000,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,16785,1310.0,0,NaN,4,35,11543,1060,436,208
3,19904,2008-08-21 13:37:58.600000000,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,49130,1288.0,1,NaN,18,35,11543,1060,436,208
4,26311,2008-08-25 16:01:58.660000000,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,155132,2644.0,1,NaN,43,35,11543,1060,436,208


In [19]:
len(merged_A_Q_U)

1503487

In [20]:
merged_A_Q_U2 = pd.merge(merged_A_Q, users,  left_on='OwnerUserId_A', right_on='Id', how='left')
merged_A_Q_U2.head()

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,ViewCount,OwnerUserId_Q,CommentCount_Q,ClosedDate_Q,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes
0,127,2008-08-01 16:13:47.600000000,37.0,35.0,2.0,NaN,126,127,1,126,...,6357,58.0,0,2012-05-08 18:11:27,2,35.0,11543.0,1060.0,436.0,208.0
1,128,2008-08-01 16:19:04.283000000,7.0,26.0,2.0,NaN,126,128,0,126,...,6357,58.0,0,2012-05-08 18:11:27,2,26.0,8520.0,2843.0,564.0,67.0
2,139,2008-08-01 16:43:30.880000000,5.0,72.0,0.0,NaN,126,139,0,126,...,6357,58.0,0,2012-05-08 18:11:27,2,72.0,18882.0,1118.0,1168.0,21.0
3,142,2008-08-01 16:56:55.830000000,2.0,42.0,0.0,NaN,126,142,0,126,...,6357,58.0,0,2012-05-08 18:11:27,2,42.0,13820.0,754.0,359.0,45.0
4,143,2008-08-01 17:01:27.240000000,3.0,86.0,0.0,NaN,126,143,0,126,...,6357,58.0,0,2012-05-08 18:11:27,2,86.0,3578.0,366.0,1641.0,59.0


In [21]:
len(merged_A_Q_U2)

1510812

In [22]:
merged_A_Q_U2[merged_A_Q_U2['Id'].isnull()]

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,ViewCount,OwnerUserId_Q,CommentCount_Q,ClosedDate_Q,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes
12,62045,2008-09-15 09:46:13.973000000,2.0,NaN,0.0,NaN,126,62045,0,126,...,6357,58.0,0,2012-05-08 18:11:27,2,NaN,NaN,NaN,NaN,NaN
57,2216,2008-08-05 12:50:59.020000000,1.0,NaN,0.0,NaN,2092,2216,0,2092,...,1411,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
59,9086,2008-08-12 17:58:25.630000000,0.0,NaN,0.0,NaN,2092,9086,0,2092,...,1411,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN
64,2977,2008-08-06 00:16:25.020000000,5.0,NaN,0.0,NaN,2968,2977,0,2968,...,42848,362.0,1,2014-05-13 17:04:49,3,NaN,NaN,NaN,NaN,NaN
98,45518,2008-09-05 10:34:05.583000000,1.0,NaN,1.0,NaN,3881,45518,0,3881,...,61344,122.0,3,NaN,72,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1502341,28739905,2015-02-26 10:26:40.583000000,0.0,NaN,0.0,NaN,28739520,28739905,0,28739520,...,22,3693338.0,3,NaN,0,NaN,NaN,NaN,NaN,NaN
1503357,28764091,2015-02-27 11:29:49.967000000,1.0,NaN,0.0,NaN,28763403,28764091,0,28763403,...,41,2816909.0,7,NaN,0,NaN,NaN,NaN,NaN,NaN
1504171,28781215,2015-02-28 11:32:52.213000000,1.0,NaN,0.0,NaN,28781162,28781215,0,28781162,...,39,3787809.0,2,NaN,0,NaN,NaN,NaN,NaN,NaN
1504180,28781360,2015-02-28 11:48:42.503000000,-1.0,NaN,2.0,NaN,28781218,28781360,0,28781218,...,29,1941853.0,5,NaN,0,NaN,NaN,NaN,NaN,NaN


In [23]:
badge_count = user_badge.groupby('UserId').size().reset_index(name='badgeCount')
badge_count

,UserId,badgeCount
0,1,298
1,2,72
2,3,109
3,4,165
4,5,284
...,...,...
2094762,4645953,1
2094763,4645962,1
2094764,4645964,1
2094765,4645973,1


In [24]:
merged_A_Q_U_B = pd.merge(merged_A_Q_U, badge_count, left_on='Id', right_on='UserId', how='inner')
merged_A_Q_U_B.head()

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,CommentCount_Q,ClosedDate_Q,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes,UserId,badgeCount
0,127,2008-08-01 16:13:47.600000000,37.0,35.0,2.0,NaN,126,127,1,126,...,0,2012-05-08 18:11:27,2,35,11543,1060,436,208,35,113
1,3886,2008-08-06 19:29:24.427000000,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,3,NaN,72,35,11543,1060,436,208,35,113
2,11567,2008-08-14 19:26:35.570000000,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,0,NaN,4,35,11543,1060,436,208,35,113
3,19904,2008-08-21 13:37:58.600000000,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,1,NaN,18,35,11543,1060,436,208,35,113
4,26311,2008-08-25 16:01:58.660000000,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,1,NaN,43,35,11543,1060,436,208,35,113


In [25]:
len(merged_A_Q_U_B)

1488203

In [26]:
merged_A_Q_U_B['ClosedDate_Q']

0          2012-05-08 18:11:27
1                          NaN
2                          NaN
3                          NaN
4                          NaN
                  ...         
1488198                    NaN
1488199                    NaN
1488200                    NaN
1488201                    NaN
1488202                    NaN
Name: ClosedDate_Q, Length: 1488203, dtype: object

In [27]:
merged_A_Q_U_B['Closed_Q'] = merged_A_Q_U_B['ClosedDate_Q'].notnull().astype(int)

In [28]:
merged_A_Q_U_B['Closed_Q']

0          1
1          0
2          0
3          0
4          0
          ..
1488198    0
1488199    0
1488200    0
1488201    0
1488202    0
Name: Closed_Q, Length: 1488203, dtype: int64

In [29]:
merged_A_Q_U_B.columns

Index(['Id_A', 'CreationDate_A', 'Score_A', 'OwnerUserId_A', 'CommentCount_A',
       'ClosedDate_A', 'QId', 'AId', 'accepted', 'Id_Q', 'CreationDate_Q',
       'Score_Q', 'ViewCount', 'OwnerUserId_Q', 'CommentCount_Q',
       'ClosedDate_Q', 'FavoriteCount', 'Id', 'Reputation', 'Views', 'UpVotes',
       'DownVotes', 'UserId', 'badgeCount', 'Closed_Q'],
      dtype='object')

In [30]:
merged_A_Q_U_B['CreationDate_A'] = pd.to_datetime(merged_A_Q_U_B['CreationDate_A'])
merged_A_Q_U_B['CreationDate_Q'] = pd.to_datetime(merged_A_Q_U_B['CreationDate_Q'])
merged_A_Q_U_B['timeDiffrence(Hours)'] = (merged_A_Q_U_B['CreationDate_A'] - merged_A_Q_U_B['CreationDate_Q']).dt.total_seconds() / 60 / 60 #based on hours
merged_A_Q_U_B['timeDiffrence(Hours)']

0          0.054889
1          0.048174
2          0.044603
3          0.036556
4          0.031850
             ...   
1488198    0.166627
1488199    0.549986
1488200    1.734582
1488201    0.309215
1488202    0.069180
Name: timeDiffrence(Hours), Length: 1488203, dtype: float64

In [31]:
merged_A_Q_U_B[merged_A_Q_U_B['timeDiffrence(Hours)']==0]

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours)
205237,14554596,2013-01-28 02:23:27,5.0,8123.0,1.0,NaN,14554595,14554596,1,14554595,...,1,8123,35143,2161,1937,192,8123,237,0,0.0
220013,20799711,2013-12-27 11:14:49,9.0,418556.0,5.0,NaN,20799710,20799711,1,20799710,...,0,418556,117299,25726,12690,4280,418556,295,0,0.0
1097537,11641986,2012-07-25 02:46:34,5.0,1339751.0,4.0,NaN,11641985,11641986,0,11641985,...,6,1339751,370,46,15,0,1339751,20,0,0.0
1109418,12644018,2012-09-28 16:38:24,1.0,902217.0,0.0,NaN,12644017,12644018,1,12644017,...,0,902217,1281,665,2082,39,902217,65,0,0.0
1142057,12683022,2012-10-02 00:33:43,4.0,650558.0,0.0,NaN,12683020,12683022,1,12683020,...,0,650558,1665,190,389,8,650558,81,0,0.0
1322645,19215085,2013-10-06 23:18:43,0.0,1478808.0,0.0,NaN,19215084,19215085,1,19215084,...,1,1478808,372,50,39,0,1478808,17,0,0.0


In [32]:
merged_A_Q_U_B[merged_A_Q_U_B['timeDiffrence(Hours)']<0]

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,FavoriteCount,Id,Reputation,Views,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours)
55,58612,2008-09-12 10:19:01.440,2.0,518.0,0.0,NaN,462923,58612,0,462923,...,26,518,2418,357,113,4,518,56,1,-3129.968767
5322,57134,2008-09-11 17:26:52.853,7.0,4926.0,0.0,NaN,462923,57134,0,462923,...,26,4926,39308,5729,981,176,4926,197,1,-3146.837819
7007,22253141,2014-03-07 14:42:57.793,0.0,2736496.0,4.0,NaN,22253140,22253141,0,22253140,...,0,2736496,7775,856,724,598,2736496,72,0,-0.000058
7134,23259195,2014-04-24 03:18:32.743,0.0,2736496.0,13.0,NaN,23259194,23259195,0,23259194,...,0,2736496,7775,856,724,598,2736496,72,0,-0.000071
7137,24516963,2014-07-01 18:24:20.787,1.0,2736496.0,0.0,NaN,24516962,24516963,0,24516962,...,0,2736496,7775,856,724,598,2736496,72,0,-0.000059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479990,28115735,2015-01-23 17:48:22.877,0.0,246568.0,0.0,NaN,28115734,28115735,0,28115734,...,0,246568,10067,992,724,5,246568,315,0,-0.000034
1480059,28121620,2015-01-24 02:16:17.947,0.0,1696177.0,0.0,NaN,28121619,28121620,0,28121619,...,0,1696177,22,13,0,0,1696177,6,0,-0.000015
1480934,28204737,2015-01-28 23:59:56.603,0.0,69173.0,0.0,NaN,28204736,28204737,0,28204736,...,0,69173,1225,176,3723,18,69173,33,0,-0.000110
1481100,28572784,2015-02-17 22:57:04.803,0.0,373418.0,6.0,NaN,28572783,28572784,1,28572783,...,0,373418,970,150,121,10,373418,19,0,-0.000055


In [33]:
len(merged_A_Q_U_B)

1488203

In [34]:
merged_A_Q_U_B2 = merged_A_Q_U_B[merged_A_Q_U_B['timeDiffrence(Hours)']>0]

In [35]:
len(merged_A_Q_U_B2)

1487610

In [36]:
merged_A_Q_U_B2.columns

Index(['Id_A', 'CreationDate_A', 'Score_A', 'OwnerUserId_A', 'CommentCount_A',
       'ClosedDate_A', 'QId', 'AId', 'accepted', 'Id_Q', 'CreationDate_Q',
       'Score_Q', 'ViewCount', 'OwnerUserId_Q', 'CommentCount_Q',
       'ClosedDate_Q', 'FavoriteCount', 'Id', 'Reputation', 'Views', 'UpVotes',
       'DownVotes', 'UserId', 'badgeCount', 'Closed_Q',
       'timeDiffrence(Hours)'],
      dtype='object')

In [37]:
df_answers = merged_A_Q_U_B2.groupby('OwnerUserId_A').size().reset_index(name='#answers_A')
df_answers

,OwnerUserId_A,#answers_A
0,1.0,2
1,3.0,1
2,9.0,1
3,13.0,441
4,17.0,9
...,...,...
192146,4644935.0,3
192147,4645046.0,1
192148,4645110.0,4
192149,4645296.0,1


In [38]:
sum(df_answers['#answers_A'].isnull())

0

In [39]:
df_answers_mean = merged_A_Q_U_B2.groupby('OwnerUserId_A')['accepted'].mean().reset_index(name='accepted_ratio_A')
df_answers_mean

,OwnerUserId_A,accepted_ratio_A
0,1.0,0.000000
1,3.0,0.000000
2,9.0,0.000000
3,13.0,0.444444
4,17.0,0.111111
...,...,...
192146,4644935.0,0.333333
192147,4645046.0,0.000000
192148,4645110.0,0.250000
192149,4645296.0,0.000000


In [40]:
merged_df = pd.merge(df_answers, df_answers_mean,  left_on='OwnerUserId_A', right_on='OwnerUserId_A', how='left')
merged_df

,OwnerUserId_A,#answers_A,accepted_ratio_A
0,1.0,2,0.000000
1,3.0,1,0.000000
2,9.0,1,0.000000
3,13.0,441,0.444444
4,17.0,9,0.111111
...,...,...,...
192146,4644935.0,3,0.333333
192147,4645046.0,1,0.000000
192148,4645110.0,4,0.250000
192149,4645296.0,1,0.000000


In [41]:
merged_A_Q_U_B2.columns

Index(['Id_A', 'CreationDate_A', 'Score_A', 'OwnerUserId_A', 'CommentCount_A',
       'ClosedDate_A', 'QId', 'AId', 'accepted', 'Id_Q', 'CreationDate_Q',
       'Score_Q', 'ViewCount', 'OwnerUserId_Q', 'CommentCount_Q',
       'ClosedDate_Q', 'FavoriteCount', 'Id', 'Reputation', 'Views', 'UpVotes',
       'DownVotes', 'UserId', 'badgeCount', 'Closed_Q',
       'timeDiffrence(Hours)'],
      dtype='object')

In [42]:
len(merged_A_Q_U_B2)

1487610

In [43]:
merged_A_Q_U_B2_R = pd.merge(merged_A_Q_U_B2, merged_df,  left_on='OwnerUserId_A', right_on='OwnerUserId_A', how='inner')
merged_A_Q_U_B2_R

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,Reputation,Views,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours),#answers_A,accepted_ratio_A
0,127,2008-08-01 16:13:47.600,37.0,35.0,2.0,NaN,126,127,1,126,...,11543,1060,436,208,35,113,1,0.054889,5,0.6
1,3886,2008-08-06 19:29:24.427,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,11543,1060,436,208,35,113,0,0.048174,5,0.6
2,11567,2008-08-14 19:26:35.570,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,11543,1060,436,208,35,113,0,0.044603,5,0.6
3,19904,2008-08-21 13:37:58.600,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,11543,1060,436,208,35,113,0,0.036556,5,0.6
4,26311,2008-08-25 16:01:58.660,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,11543,1060,436,208,35,113,0,0.031850,5,0.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487605,28922535,2015-03-08 02:48:19.857,0.0,4645692.0,2.0,NaN,28922468,28922535,0,28922468,...,1,1,0,0,4645692,3,0,0.166627,1,0.0
1487606,28922813,2015-03-08 03:40:11.950,0.0,3313892.0,0.0,NaN,28922649,28922813,0,28922649,...,1,0,0,0,3313892,1,0,0.549986,1,0.0
1487607,28922747,2015-03-08 03:24:05.497,0.0,4643110.0,2.0,NaN,28922175,28922747,0,28922175,...,1,0,0,0,4643110,2,0,1.734582,2,0.0
1487608,28922923,2015-03-08 04:00:36.173,0.0,4643110.0,0.0,NaN,28922819,28922923,0,28922819,...,1,0,0,0,4643110,2,0,0.309215,2,0.0


In [44]:
# Function to calculate the new column to exclude the current smaple from accepted_ratio_A

def calculate_value(row):
  if  row['#answers_A'] < 2 :
    return np.nan
  else :
    if row['accepted'] == 1:
        return ((row['accepted_ratio_A'] * row['#answers_A']) - 1) / (row['#answers_A'] - 1)
    else:
        return (row['accepted_ratio_A'] * row['#answers_A']) / (row['#answers_A'] - 1)

# Apply the function to create the new column 'new_column'
merged_A_Q_U_B2_R['corrected_accepted_ratio_A'] = merged_A_Q_U_B2_R.apply(lambda row: calculate_value(row), axis=1)

In [45]:
merged_A_Q_U_B2_R

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,Views,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours),#answers_A,accepted_ratio_A,corrected_accepted_ratio_A
0,127,2008-08-01 16:13:47.600,37.0,35.0,2.0,NaN,126,127,1,126,...,1060,436,208,35,113,1,0.054889,5,0.6,0.50
1,3886,2008-08-06 19:29:24.427,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,1060,436,208,35,113,0,0.048174,5,0.6,0.50
2,11567,2008-08-14 19:26:35.570,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,1060,436,208,35,113,0,0.044603,5,0.6,0.75
3,19904,2008-08-21 13:37:58.600,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,1060,436,208,35,113,0,0.036556,5,0.6,0.50
4,26311,2008-08-25 16:01:58.660,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,1060,436,208,35,113,0,0.031850,5,0.6,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487605,28922535,2015-03-08 02:48:19.857,0.0,4645692.0,2.0,NaN,28922468,28922535,0,28922468,...,1,0,0,4645692,3,0,0.166627,1,0.0,NaN
1487606,28922813,2015-03-08 03:40:11.950,0.0,3313892.0,0.0,NaN,28922649,28922813,0,28922649,...,0,0,0,3313892,1,0,0.549986,1,0.0,NaN
1487607,28922747,2015-03-08 03:24:05.497,0.0,4643110.0,2.0,NaN,28922175,28922747,0,28922175,...,0,0,0,4643110,2,0,1.734582,2,0.0,0.00
1487608,28922923,2015-03-08 04:00:36.173,0.0,4643110.0,0.0,NaN,28922819,28922923,0,28922819,...,0,0,0,4643110,2,0,0.309215,2,0.0,0.00


In [46]:
mean_value  = merged_A_Q_U_B2_R['corrected_accepted_ratio_A'].mean()

In [47]:
# Fill null values in 'corrected_accepted_ratio_A' with mean

mean_value  = merged_A_Q_U_B2_R['corrected_accepted_ratio_A'].mean()
merged_A_Q_U_B2_R['corrected_accepted_ratio_A'].fillna(round(mean_value, 2), inplace=True)
merged_A_Q_U_B2_R

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,Views,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours),#answers_A,accepted_ratio_A,corrected_accepted_ratio_A
0,127,2008-08-01 16:13:47.600,37.0,35.0,2.0,NaN,126,127,1,126,...,1060,436,208,35,113,1,0.054889,5,0.6,0.50
1,3886,2008-08-06 19:29:24.427,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,1060,436,208,35,113,0,0.048174,5,0.6,0.50
2,11567,2008-08-14 19:26:35.570,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,1060,436,208,35,113,0,0.044603,5,0.6,0.75
3,19904,2008-08-21 13:37:58.600,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,1060,436,208,35,113,0,0.036556,5,0.6,0.50
4,26311,2008-08-25 16:01:58.660,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,1060,436,208,35,113,0,0.031850,5,0.6,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487605,28922535,2015-03-08 02:48:19.857,0.0,4645692.0,2.0,NaN,28922468,28922535,0,28922468,...,1,0,0,4645692,3,0,0.166627,1,0.0,0.31
1487606,28922813,2015-03-08 03:40:11.950,0.0,3313892.0,0.0,NaN,28922649,28922813,0,28922649,...,0,0,0,3313892,1,0,0.549986,1,0.0,0.31
1487607,28922747,2015-03-08 03:24:05.497,0.0,4643110.0,2.0,NaN,28922175,28922747,0,28922175,...,0,0,0,4643110,2,0,1.734582,2,0.0,0.00
1487608,28922923,2015-03-08 04:00:36.173,0.0,4643110.0,0.0,NaN,28922819,28922923,0,28922819,...,0,0,0,4643110,2,0,0.309215,2,0.0,0.00


In [48]:
df = merged_A_Q_U_B2_R.groupby('QId').size().reset_index(name='#answers_Q')
df

,QId,#answers_Q
0,123,17
1,126,18
2,382,1
3,564,15
4,2092,3
...,...,...
717670,28922732,1
717671,28922734,1
717672,28922819,5
717673,28922872,2


In [49]:
merged_A_Q_U_B2_R2 = pd.merge(merged_A_Q_U_B2_R, df,  left_on='QId', right_on='QId', how='left')
merged_A_Q_U_B2_R2

,Id_A,CreationDate_A,Score_A,OwnerUserId_A,CommentCount_A,ClosedDate_A,QId,AId,accepted,Id_Q,...,UpVotes,DownVotes,UserId,badgeCount,Closed_Q,timeDiffrence(Hours),#answers_A,accepted_ratio_A,corrected_accepted_ratio_A,#answers_Q
0,127,2008-08-01 16:13:47.600,37.0,35.0,2.0,NaN,126,127,1,126,...,436,208,35,113,1,0.054889,5,0.6,0.50,18
1,3886,2008-08-06 19:29:24.427,133.0,35.0,1.0,NaN,3881,3886,1,3881,...,436,208,35,113,0,0.048174,5,0.6,0.50,21
2,11567,2008-08-14 19:26:35.570,1.0,35.0,0.0,NaN,11561,11567,0,11561,...,436,208,35,113,0,0.044603,5,0.6,0.75,4
3,19904,2008-08-21 13:37:58.600,135.0,35.0,0.0,NaN,19893,19904,1,19893,...,436,208,35,113,0,0.036556,5,0.6,0.50,13
4,26311,2008-08-25 16:01:58.660,20.0,35.0,7.0,NaN,26305,26311,0,26305,...,436,208,35,113,0,0.031850,5,0.6,0.75,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1487605,28922535,2015-03-08 02:48:19.857,0.0,4645692.0,2.0,NaN,28922468,28922535,0,28922468,...,0,0,4645692,3,0,0.166627,1,0.0,0.31,2
1487606,28922813,2015-03-08 03:40:11.950,0.0,3313892.0,0.0,NaN,28922649,28922813,0,28922649,...,0,0,3313892,1,0,0.549986,1,0.0,0.31,2
1487607,28922747,2015-03-08 03:24:05.497,0.0,4643110.0,2.0,NaN,28922175,28922747,0,28922175,...,0,0,4643110,2,0,1.734582,2,0.0,0.00,1
1487608,28922923,2015-03-08 04:00:36.173,0.0,4643110.0,0.0,NaN,28922819,28922923,0,28922819,...,0,0,4643110,2,0,0.309215,2,0.0,0.00,5


In [ ]:
# Grouping the DataFrame by 'QId'
grouped = merged_A_Q_U_B2_R.groupby('QId')

result_dict = {}
# Iterating through each group
for qid, group in grouped:
    for index, row in group.iterrows():
      current_id = row['Id_A']
      current_time_diff = row['timeDiffrence(Hours)']

      # Counting the number of other Id_A with less time difference for the current Id_A
      count = sum(group['timeDiffrence(Hours)'] < current_time_diff)

      # Storing the count for the current Id_A
      result_dict[current_id] = count


In [ ]:
len(result_dict)

In [ ]:
result_df = pd.DataFrame(list(result_dict.items()), columns=['Id_A', 'Less_Time_Diff_Count'])
result_df

In [ ]:
merged_A_Q_U_B2_R2.columns

In [ ]:
merged_A_Q_U_B2_R3 = pd.merge(merged_A_Q_U_B2_R2, result_df,  left_on='Id_A', right_on='Id_A', how='inner')
merged_A_Q_U_B2_R3

In [ ]:
df_Q = merged_A_Q_U_B2_R3.groupby('OwnerUserId_Q').size().reset_index(name='#answers_Q_U')
df_Q

In [ ]:
df_Q_mean = merged_A_Q_U_B2_R3.groupby('OwnerUserId_Q')['accepted'].mean().reset_index(name='accepted_ratio_Q')
df_Q_mean

In [ ]:
merged_df = pd.merge(df_Q, df_Q_mean,  left_on='OwnerUserId_Q', right_on='OwnerUserId_Q', how='left')
merged_df

In [ ]:
merged_A_Q_U_B2_R4 = pd.merge(merged_A_Q_U_B2_R3, merged_df,  left_on='OwnerUserId_Q', right_on='OwnerUserId_Q', how='inner')
merged_A_Q_U_B2_R4

In [ ]:
# Function to calculate the new column to exclude the current smaple from accepted_ratio_A

def calculate_value(row):
  if  row['#answers_Q_U'] < 2 :
    return np.nan
  else :
    if row['accepted'] == 1:
        return ((row['accepted_ratio_Q'] * row['#answers_Q_U']) - 1) / (row['#answers_Q_U'] - 1)
    else:
        return (row['accepted_ratio_Q'] * row['#answers_Q_U']) / (row['#answers_Q_U'] - 1)

# Apply the function to create the new column 'new_column'
merged_A_Q_U_B2_R4['corrected_accepted_ratio_Q'] = merged_A_Q_U_B2_R4.apply(lambda row: calculate_value(row), axis=1)

In [ ]:
merged_A_Q_U_B2_R4

In [ ]:
mean_value  = merged_A_Q_U_B2_R4['corrected_accepted_ratio_Q'].mean()
mean_value

In [ ]:
# Fill null values in 'corrected_accepted_ratio_A' with mean

mean_value  = merged_A_Q_U_B2_R4['corrected_accepted_ratio_Q'].mean()
merged_A_Q_U_B2_R4['corrected_accepted_ratio_Q'].fillna(round(mean_value, 2), inplace=True)
merged_A_Q_U_B2_R4

In [ ]:
tags

In [ ]:
questions.columns

In [ ]:
len(tags['Id'].unique())

In [ ]:
len(questions['Id'].unique())

In [ ]:
merged_Q_A = pd.merge(questions, Q_A,  left_on='Id', right_on='QId', how='inner')
merged_Q_A

In [ ]:
merged_Q_A_tag = pd.merge(tags, merged_Q_A,  left_on='Id', right_on='Id', how='left')
merged_Q_A_tag

In [ ]:
df_tag = merged_Q_A_tag.groupby('Tags')['accepted'].mean().reset_index(name='accepted_ratio_tag')
df_tag

In [ ]:
df_tag2 = merged_Q_A_tag.groupby('Tags').size().reset_index(name='#tag')
df_tag2

In [ ]:
merged_tag = pd.merge(df_tag, df_tag2,  left_on='Tags', right_on='Tags', how='inner')
merged_tag.loc[merged_tag['#tag']<10, 'accepted_ratio_tag'] = np.nan   #we ignore ratio for tags with few occurrence
merged_tag

In [ ]:
mean_value = merged_tag['accepted_ratio_tag'].mean()
merged_tag['accepted_ratio_tag'].fillna(round(mean_value, 2), inplace=True)
merged_tag

In [ ]:
merged_Q_A_tag2 = pd.merge(merged_Q_A_tag, merged_tag,  left_on='Tags', right_on='Tags', how='left')
merged_Q_A_tag2

In [ ]:
df2 = merged_Q_A_tag2.groupby('QId')['accepted_ratio_tag'].mean().reset_index(name='accepted_ratio_tags_mean')
df2

In [ ]:
df3 = tags.groupby('Id').size().reset_index(name='#tags_Q')
df3

In [ ]:
merged_df = pd.merge(df2, df3,  left_on='QId', right_on='Id', how='inner')
merged_df

In [ ]:
merged_A_Q_U_B2_R5 = pd.merge(merged_A_Q_U_B2_R4, merged_df,  left_on='QId', right_on='QId', how='left')
merged_A_Q_U_B2_R5

In [ ]:
merged_A_Q_U_B2_R5.columns

In [ ]:
merged_A_Q_U_B2_R6 = merged_A_Q_U_B2_R5.drop(columns=['Id_A','CreationDate_A','OwnerUserId_A','ClosedDate_A','QId','AId','Id_Q','CreationDate_Q','OwnerUserId_Q','ClosedDate_Q','Id_x','UserId','#answers_A','accepted_ratio_A','#answers_Q_U','accepted_ratio_Q','Id_y'], inplace=False)
merged_A_Q_U_B2_R6

In [ ]:
merged_A_Q_U_B2_R6.columns

In [ ]:
# Rename columns
merged_A_Q_U_B2_R6 = merged_A_Q_U_B2_R6.rename(columns={'ViewCount': 'ViewCount_Q', 'FavoriteCount': 'FavoriteCount_Q', 'Views': 'Views_U', 'timeDiffrence(Hours)': 'Time_Diffrence(H)','Less_Time_Diff_Count': 'Earlier_Answers', 'corrected_accepted_ratio_A': 'Accepted_Ratio_A','corrected_accepted_ratio_Q': 'Accepted_Rratio_Q'})

print(merged_A_Q_U_B2_R6)

In [ ]:
len(merged_A_Q_U_B2_R6[merged_A_Q_U_B2_R6['accepted']==1])

In [ ]:
len(merged_A_Q_U_B2_R6[merged_A_Q_U_B2_R6['accepted']==0])

In [ ]:
merged_A_Q_U_B2_R6.corr()['accepted']

In [ ]:

# Calculate correlation matrix
correlation_matrix = merged_A_Q_U_B2_R6.corr()

fig = plt.figure(figsize=(15,15))

fig.patch.set_facecolor('None')
#fig.patch.set_alpha(0.7)

heatmap = sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap (Questions)')

# Set the background of the plot to be transparent
heatmap.patch.set_facecolor('none')
heatmap.patch.set_alpha(0.1)
plt.show()

In [ ]:
Accepted_df = merged_A_Q_U_B2_R6.drop(columns=['#answers_Q'], inplace=False)
Accepted_df.columns

# Create dataset splits

In [ ]:
X = Accepted_df.drop(["accepted"], axis=1)
y = Accepted_df["accepted"]

## Evaluate best model

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'learning_rate': [0.1,0.05],
    'max_depth': [12, 15, 18],
    'n_estimators': [350, 400, 450],
}

# Define the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method='hist',
    use_label_encoder=False,
    nthread=-1,
    seed=42,
)

grid_search = GridSearchCV(estimator=xgb_classifier, param_grid=param_grid, scoring='accuracy', cv=2, verbose=2)
grid_search.fit(X, y)

best_params = grid_search.best_params_
print('Best Hyperparameters:', best_params)

# Training

In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer,accuracy_score, precision_score, recall_score, f1_score

# Define the XGBoost model
xgb_model = xgb.XGBClassifier(learning_rate=0.1, max_depth=15, n_estimators=400, random_state=42)

# Define cross-validation strategy (e.g., using Stratified K-Fold with 10 folds)
cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)

# Define custom scoring metrics
metrics = ['accuracy', 'precision', 'recall', 'f1']

# Calculate multiple evaluation metrics using cross-validation
cv_results  = cross_validate(xgb_model, X, y, cv=cv, scoring=metrics, return_estimator=True)

# Print the results
print(cv_results)


# Evalutation

In [ ]:
# Print the mean scores for each metric
print("Accuracy mean score:", cv_results['test_accuracy'].mean())
print("Precision mean score:", cv_results['test_precision'].mean())
print("Recall mean score:", cv_results['test_recall'].mean())
print("f1-score:", cv_results['test_f1'].mean())

In [ ]:
xgb.plot_importance(cv_results['estimator'][0])

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(features, labels, test_size=0.2, random_state=42)

In [ ]:
train_X.columns

# Model (XGBOOST)

In [ ]:
# Define the XGBoost classifier
xgb_classifier = xgb.XGBClassifier(
    objective="binary:logistic",
    eval_metric="logloss",
    tree_method='hist',
    use_label_encoder=False,
    nthread=-1,
    seed=42,
)


In [ ]:
# Train the XGBoost classifier
xgb_classifier.fit(train_X, train_y)

In [ ]:
# Make predictions on the validation set
val_predictions = xgb_classifier.predict(val_X)

# Evaluate the performance on the validation set
accuracy = metrics.accuracy_score(val_y, val_predictions)
print(f'Accuracy on the validation set: {accuracy:.2f}')


accuracy = metrics.f1_score(val_y, val_predictions)
print(f'F1 on the validation set: {accuracy:.2f}')

accuracy = metrics.precision_score(val_y, val_predictions)
print(f'Precision on the validation set: {accuracy:.2f}')

accuracy = metrics.recall_score(val_y, val_predictions)
print(f'Recall on the validation set: {accuracy:.2f}')

# Print classification report for more detailed evaluation
print('\nClassification Report:\n', metrics.classification_report(val_y, val_predictions))

# Feature importance

In [ ]:
import matplotlib.pyplot as plt

xgb.plot_importance(xgb_classifier)
plt.show()

# Hyperparameter search (GridSearch)

In [ ]:
# Make predictions on the validation set
val_predictions = best_xgb_classifier.predict(val_X)

# Evaluate the performance on the validation set
accuracy = metrics.accuracy_score(val_y, val_predictions)
print(f'Accuracy on the validation set: {accuracy:.2f}')


accuracy = metrics.f1_score(val_y, val_predictions)
print(f'F1 on the validation set: {accuracy:.2f}')

accuracy = metrics.precision_score(val_y, val_predictions)
print(f'Precision on the validation set: {accuracy:.2f}')

accuracy = metrics.recall_score(val_y, val_predictions)
print(f'Recall on the validation set: {accuracy:.2f}')

# Print classification report for more detailed evaluation
print('\nClassification Report:\n', metrics.classification_report(val_y, val_predictions))